# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [8]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug
# Only needed locally.
# !pip install sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 3.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 1.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [7]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import zipfile
import os
import pandas as pd
import glob

In [22]:
# TODO: Get some common variables
# sagemaker client runtime or otherwise?
# default or specific bucket?
# others in course 2 project - check what other common things were done

bucket = "course3-project"
execution_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

print(f"Chosen Bucket: {bucket}")
print(f"Execution role: {execution_role}")
print(f"Sagemaker Session: {sagemaker_session}")
print(f"Sagemaker Session Region: {region}")

Chosen Bucket: course3-project
Execution role: arn:aws:iam::569113498951:role/service-role/AmazonSageMaker-ExecutionRole-20221005T103614
Sagemaker Session: <sagemaker.session.Session object at 0x7f97c4d3f5d0>
Sagemaker Session Region: us-east-1


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
# Upload? This was already there using wget - but I decided to use my own data set. Is using the dog data set better than mine?
# Comment out below because I have already uploaded cell data to s3 using the Web Console
# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

# But add code to fetch zip from s3, extract it and upload it in a format required for training
# Will need to extract data in s3 and upload it in appropriate format to be used by training script
# This extract portion of the ETL need not be performed in the Data class of the trianing script
# can extract locally and then push to s3 (web console) or extract in s3 itself (extract on s3) or in this notebook and push to s3 or submission script?
# what is the submission script - this could be considered submission script? what is inference script?
# need to also save the labels on the images - can do on the file name or create a separate metadata annotation file which is processd by training script
# or can just send as pickled files? Do that which matches the format for other Data classes in pytorch - e.g. CIFAR10
# might need to do either of the above ways here
# once of vs recurring data tasks - should not be done in training script if once off? new images vs old images - may need to do it here again for new images e.g. pickle again or generate annotation files again etc - but not part of the training script right?
# what should be in training script and what not!!!
# I think best done before trinaig script!!!

In [13]:
# May not need thses paths - can directly upload from the zipped folder which was unzipped - it has a folder structure corresponding to train test etc.
# Do we have a validation data set?
# Will we be using one?
# Local Data Paths
train_dir = os.path.join(os.getcwd(), "data/train")
validation_dir = os.path.join(os.getcwd(), "data/validation")
test_dir = os.path.join(os.getcwd(), "data/test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
# !mkdir ./data
print(os.getcwd())

!rm -r ./data/train
!rm -r ./data/validation
!rm -r ./data/test

/root/Course3/CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter


In [11]:
# Data Paths in s3
s3 = boto3.client('s3')
s3.download_file(bucket, "data/dataset2-master.zip", "./data/dataset2-master.zip")

In [12]:
# Extract
# This is taking a long time to do
# The time it takes for data pre-processing vs training time should be looked at differently
# Will need different kinds of instances (type and count) based on workload and how large it is
# These are 2 different things. So ideally should not be on same script and should use different instances
# The training can use a job while this can be done here in the notebook!!! Even this though can be done on another
# Processing sagemaker job - Try it out!!! How does it feed into training job!!! Check old course
# Also it seems the jobs cost more - in house notebook instnces for data science are not too expensive maybe on studio
# The only preprocessing which should be done the training script should be that which is dynamic and/or randimised e.g after some cropping or random scaling or re-ordering of the bathes on load etc
def extract_blood_cell_data(path_to_zip_file):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall("./data")
extract_blood_cell_data("./data/dataset2-master.zip")

In [19]:
train_neutrophil=glob.glob("./data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL" + '/**/*.jpeg', recursive=True)
train_monocyte=glob.glob("./data/dataset2-master/dataset2-master/images/TRAIN/MONOCYTE" + '/**/*.jpeg', recursive=True)
train_lymphocyte=glob.glob("./data/dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE" + '/**/*.jpeg', recursive=True)
train_eosinophil=glob.glob("./data/dataset2-master/dataset2-master/images/TRAIN/EOSINOPHIL" + '/**/*.jpeg', recursive=True)

print(len(train_neutrophil))
print(len(train_monocyte))
print(len(train_lymphocyte))
print(len(train_eosinophil))

# Not sure if I should do the validation split here or in pandas
# What is the best length for these datasets I also do not know

# validation_neutrophil = train_neutrophil[int(len(train_neutrophil)*0.80):]
# train_neutrophil = train_neutrophil[:int(len(train_neutrophil)*0.80)]
# validation_monocyte = train_monocyte[int(len(train_monocyte)*0.80):]
# train_monocyte = train_monocyte[:int(len(train_monocyte)*0.80)]
# validation_lymphocyte = train_lymphocyte[int(len(train_lymphocyte)*0.80):]
# train_lymphocyte = train_lymphocyte[:int(len(train_lymphocyte)*0.80)]
# validation_eosinophil = train_eosinophil[int(len(train_eosinophil)*0.80):]
# train_eosinophil = train_eosinophil[:int(len(train_eosinophil)*0.80)]

# print(len(train_neutrophil))
# print(len(train_monocyte))
# print(len(train_lymphocyte))
# print(len(train_eosinophil))

# print(len(validation_neutrophil))
# print(len(validation_monocyte))
# print(len(validation_lymphocyte))
# print(len(validation_eosinophil))

test_neutrophil=glob.glob("./data/dataset2-master/dataset2-master/images/TEST/NEUTROPHIL" + '/**/*.jpeg', recursive=True)
test_monocyte=glob.glob("./data/dataset2-master/dataset2-master/images/TEST/MONOCYTE" + '/**/*.jpeg', recursive=True)
test_lymphocyte=glob.glob("./data/dataset2-master/dataset2-master/images/TEST/LYMPHOCYTE" + '/**/*.jpeg', recursive=True)
test_eosinophil=glob.glob("./data/dataset2-master/dataset2-master/images/TEST/EOSINOPHIL" + '/**/*.jpeg', recursive=True)
print(len(test_neutrophil))
print(len(test_monocyte))
print(len(test_lymphocyte))
print(len(test_eosinophil))

2499
2478
2483
2497
624
620
620
623


In [25]:
# Should I sync this or numpy version?
# Very time consuming - mayb eeven expesnsive
os.environ["DEFAULT_S3_BUCKET"] = bucket
print(os.environ["DEFAULT_S3_BUCKET"])
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL s3://${DEFAULT_S3_BUCKET}/data/train
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TRAIN/MONOCYTE s3://${DEFAULT_S3_BUCKET}/data/train
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE s3://${DEFAULT_S3_BUCKET}/data/train
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TRAIN/EOSINOPHIL s3://${DEFAULT_S3_BUCKET}/data/train

!aws s3 sync ./data/dataset2-master/dataset2-master/images/TEST/NEUTROPHIL s3://${DEFAULT_S3_BUCKET}/data/test
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TEST/MONOCYTE s3://${DEFAULT_S3_BUCKET}/data/test
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TEST/LYMPHOCYTE s3://${DEFAULT_S3_BUCKET}/data/test
!aws s3 sync ./data/dataset2-master/dataset2-master/images/TEST/EOSINOPHIL s3://${DEFAULT_S3_BUCKET}/data/test

course3-project
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_2400.jpeg to s3://course3-project/data/train/_0_2400.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_1986.jpeg to s3://course3-project/data/train/_0_1986.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_292.jpeg to s3://course3-project/data/train/_0_292.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_4888.jpeg to s3://course3-project/data/train/_0_4888.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_7337.jpeg to s3://course3-project/data/train/_0_7337.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_4892.jpeg to s3://course3-project/data/train/_0_4892.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_1095.jpeg to s3://course3-project/data/train/_0_1095.jpeg
upload: data/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL/_0_9495

In [34]:
# Should I preprocess this data in any way using image processing techniques? see example notebooks on kaggle
# but not interested in the data science part of it - but can check for in case
# is there a need to create annotated file? - or just append the label to the file name.

# 4 Class Directories
# Train and Test
# And Also added Validation without a directory - will need to identify by file name?
# Then won't be able to use sync command...
# need to create its own directory then
# and be able to sync from it for each class ?
# rather better to not go here and training code takes care...
# dynamic and randomised

# Should have not uplaoded everything since time consuming - only some images... should train little

df = pd.DataFrame(columns = ['ImageFileName', 'Class'])
for i in range(len(train_neutrophil)):
    df = df.append({'ImageFileName': train_neutrophil[i].split("/")[-1], 'Class': '0'}, ignore_index=True)

for i in range(len(train_monocyte)):
    df = df.append({'ImageFileName': train_monocyte[i].split("/")[-1], 'Class': '1'}, ignore_index=True)
    
for i in range(len(train_lymphocyte)):
    df = df.append({'ImageFileName': train_lymphocyte[i].split("/")[-1], 'Class': '2'}, ignore_index=True)
    
for i in range(len(train_eosinophil)):
    df = df.append({'ImageFileName': train_eosinophil[i].split("/")[-1], 'Class': '3'}, ignore_index=True)

df.to_csv("train.lst", header=False)

# use concat instead
df = pd.DataFrame(columns = ['ImageFileName', 'Class'])
for i in range(len(test_neutrophil)):
    df = df.append({'ImageFileName': test_neutrophil[i].split("/")[-1], 'Class': '0'}, ignore_index=True)

for i in range(len(test_monocyte)):
    df = df.append({'ImageFileName': test_monocyte[i].split("/")[-1], 'Class': '1'}, ignore_index=True)
    
for i in range(len(test_lymphocyte)):
    df = df.append({'ImageFileName': test_lymphocyte[i].split("/")[-1], 'Class': '2'}, ignore_index=True)
    
for i in range(len(test_eosinophil)):
    df = df.append({'ImageFileName': test_eosinophil[i].split("/")[-1], 'Class': '3'}, ignore_index=True)

df.to_csv("test.lst", header=False)
print(df)

# separate training script for aws vs local - aws will also fetch the data?
# from s3? - not in a public location - my s3 bucket in my account

# lst file etc is about how we planning to read the data on our side - which folder it saves it to - the estimator saves it to a folder on the instance
# environement variable -parser argument - input channel
# when running training job the logs show this!!! the env variable

# saving data to s3 in a way that will be easily loadable by the Data class and minimal work for transformers and loaders - which fits the design pattern
# numpy like format? maybe even as pickle files with numpy format to use? - any advantage?
# or disadvantage to pickle files?
# we only need to the image for any data exploration and visualisatiosn which we should ideally not do on the training script? - Check how CIFAR works in PYTORCH - the soruce implementation
# since numpy is what most models prefer - since numpy can easily be changed to tensors which our chosen pytorhc framwerk can use?
# check tutorial I am using and project 2 to see best ways to do it and course souham videos and idoms
# clear disticinction between train and test
# generating a dataset
# train validation split
# seems like it will fetch input channels all of them in the directory? for script mode - for geneirc estimator not sure?
# how to supress output when saving and pusing
# how to supress outputwhen viewing
# READ THIS CELL NICELY
# MAYBE PICKLE THE DATA FIRST
# LETS DO THROUGH A PROCESSING JOB
# DO WE NEED ANOTATION FILE + DATA, VS PICKLED FILE VS NONE AND ALL DONE IN TRAINING JOB
# READ IMAGE IN PIL FORMAT OR OTHER
# IS PICKEL EXPENSIVE TO TRANSFER AND LARGE IN SIZE?
# WHERE WILL OUTPUT OF PROCESSING JOB GO
# SOMETHING ELSE FORGOT to do or think
# DO IT like this: https://www.cs.toronto.edu/~kriz/cifar.html format - check project 2!!!
# MAYBE COMPLETE THE OTHER PARTS FIRST BELOW
# Will need something like lambdas and step functions maybe to orchestrate these jobs!!! outside this kernel


      ImageFileName Class
0     _29_8380.jpeg     0
1      _15_879.jpeg     0
2     _32_6577.jpeg     0
3      _5_3600.jpeg     0
4       _31_73.jpeg     0
...             ...   ...
2482  _11_9684.jpeg     3
2483   _7_2893.jpeg     3
2484   _0_6052.jpeg     3
2485  _11_3170.jpeg     3
2486   _6_4766.jpeg     3

[2487 rows x 2 columns]


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, IntegerParameter, HyperparameterTuner
#TODO: Declare your HP ranges, metrics etc.
ranges = [{"lr": ContinuousParameter(0.001, 0.1), "batch_size": CategoricalParameter([32, 64]), "epochs":IntegerParameter(2, 4)}]
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import HyperparameterTuner

#TODO: Create estimators for your HPs

estimator = PyTorch(
    base_job_name="PyTorchEstimatorHyperParameterTuningEstimatorJob",
    source_dir="CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter",
    entry_point="hpo.py",
    instance_type="",
    instance_count=1,
    framework_version="0.8",
    python_version="py36",
    role=role)
# TODO: Your estimator here

tuner = HyperparameterTuner(
    estimator=estimator,
    base_tuning_job_name="PyTorchEstimatorHyperParameterTuningJob",
    hyperparameter_ranges=ranges,
    role=role,
    objective_type=objective_type,
    metric_definitions=metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2
)
# TODO: Your HP tuner here

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit() # TODO: Remember to include your data channels

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()
#TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import Rule, ProfilerRule, DebuggerHookConfig, ProfilerConfig, rule_configs
rules = [Rule(rule_configs.poor_weight_initialization()),
       Rule(rule_configs.all_zero()),
       Rule(rule_configs.exploding_tensor()),
       Rule(rule_configs.loss_not_decreasing()),
       ProfilerRule(rule_configs.ProfilerReport())]
debugger_config = DebuggerHookConfig(s3_output_path="")
profiler_config = ProfilerConfig(s3_output_path="", system_monitor_interval_millis="", framework_profile_params="")

In [ ]:
from sagemaker.pytorch import PyTorch
# TODO: Create and fit an estimator

estimator = PyTorch(base_job_name="DebuggingFineTunedModelJob",
                    source_dir="",
                    entry_point="train_model.py",
                    role=role,
                    instance_type="",
                    instance_count=1,
                    framework_version="",
                    python_version="",
                    hyper_parameters="",
                    rules=rules,
                    profiler_config=profiler_config,
                    debugger_hook_config=debugger_config
)
# TODO: Your estimator here

estimator.fit(inputs="")

In [ ]:
estimator.latest_training_job.job_name
estimator.output_path
estimator.latest_job_profiler_artificats_path()
estimator.lateste_job_debuggin_artificats_path()

# TODO: Plot a debugging output.
# What exactly needs to be plotted
from smdebug.trials import create_trial
trial1 = create_trial(path="s3", name="DebuggingTrial")


**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()